**Step 1:** Calculate the equivalent rainfall height, Hr based on the ground station latitude (θ).

θ=ground station latitude

If Hr–Hs is less than or equal to zero, the predicted rain attenuation for any time percentage is zero
and the following steps are not required.

For areas of the world where no specific information is available, the mean annual 0° C
isotherm height above mean sea level, h 0 , is an integral part of this Recommendation and is
available in the form of a digital map provided in the file R-REC-P.839-4-201309-I!!ZIP-E
In this section, obtain the mean annual rain height above mean sea level, hR as given in Recommendation ITU-R P.839-4.
may be obtained from the 0° C
isotherm as:
h R = h0 + 0.36 km
The data is provided from 0° to 360° in longitude and from +90° to –90° in latitude. For a location
different from the gridpoints, the mean annual 0° C isotherm height above mean sea level at the
desired location can be derived by performing a bilinear interpolation on the values at the four
closest gridpoints.

**Step 2:** Determine the slant path length, Ls below the rainfall height
Where *elev* is the minimum elevation angle of the ground station and *Hs* the height above the sea level of the ground station. 

**Step 3:** Compute the horizontal projection, LG of the Ls.

**Step 4:** Obtain rain intensity exceeded for 0.01%, R0.01 (mm/h) and use it to calculate the specific attenuation, γ0.01(dB/km).

*k* and *alp* depend on the raindrop size distribution, frequency, polarization and rain temperature. They are obtained from ITU-R P.618-8 recommendation document

**Step 5:** Establish the horizontal path adjustment factor, *hrf*, for 0.01% time by:
Where f=transmit frequency (Hz)

**Step 6:** Compute the adjusted rainy path length, LR:
1. First calculate the tau

If tau is **greater than** elevation angle (elev)

If tau is **less than or equal** to elevation angle (elev)

**Step 7:** Calculated the vertical reduction factor, *rv* for 0.01% of the time

**Step 8:** Derive effective path length, Le (km) by:

**Step 9:** Calculate the predicated attenuation, *AP* above 0.01% for the year by

**Step 10:** The attenuation can then be approximated for other percentages exceeded in the a range  0.001% -10% may be estimated for 0.01% exceeded during the year as:

if elevation angle **is equal or greater than 25** and station latitude **is less than** 36

if elevation angle **is less than 25** and station latitude **is less than** 36

If elevation angle is equal or greater that 36 degrees or the probability is equal or greater than 1%


In [4]:
import math
from math import *
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Compute Equivalent rainfall height, Hr
thet=float(input("Enter the Ground Station Absolute Latitude: "))# Latitude of the ground station
h0=float(input("Enter Mean Annual 0° C Isotherm Height Above Mean Sea Level : "))
Hr=h0+0.36
print('Rainfall Height= '+str(Hr)+ 'm')
Hs=float(input("Enter the Ground Station Height above Sea Level in metres: "))
elev=float(input("Enter the Antenna Elevation Angle in degrees: ")) #Minimum elevation angle

#Calculating Slant Path, Ls (STEP TWO)
Ls=(-1*(Hr-Hs))/(math.sin(math.radians(elev)))
print('Slant Path Length= '+str(Ls)+ 'm')

#Calculating Horizontal Projection LG (STEP THREE)
LG=Ls*(math.cos(math.radians(elev)))

#Determining Specific Attenuation (STEP FOUR)
R=float(input("Enter 1-min rain rate exceeded for 0.01% of an average year from ITU-R P.837-7, raingauge or Satellite: "))
k=float(input("Enter Coefficient k from RECOMMENDATION ITU-R P.838-3 : ")) 
alp=float(input("Enter Coefficient alpha from RECOMMENDATION ITU-R P.838-3 : "))
gamR=k*R**alp   #in dB/km
print('Specific Attenuation= '+str(gamR)+ 'dB/km')

#Calculate Horizontal Adjustment Factor (STEP FIVE)
fg=float(input("Enter the Transmission Frequency in GHz: "))
f=fg*10**9   #Uplink frequency
st=0.38*(1-(math.exp(-2*LG)))
ft=0.78*(math.sqrt((LG*gamR)/(f)))
hrf=(1)/((1)+(ft)-(st))
print('Horizontal Adjustement Factor= '+str(hrf))
#Calculate tau for boundary conditions
tau=degrees(atan((Hr-Hs)/(LG*hrf)))

#Now calculate the Adjusted Rainy Path Length (STEP 6)
if tau>elev:
    LR=((LG*hrf)/(math.cos(math.radians(elev))))
else:
    LR=((Hr-Hs)/(math.sin(math.radians(elev))))    
print('Adjusted Rainy Path Length= '+str(LR)+'m')

#Set boundary conditions for calculation of Vertical Adjustment Factor (STEP SEVEN)
if thet<36:
    χ=36-thet  #if thet is less than 36
else:
    χ=0        #if thet is equal or above 36
sqt=(math.sqrt(math.sin(math.radians(elev))))     #Term with sine and square
ext=ext=(31*(math.exp(-1*((elev)/(1+χ)))))      #Term with the exponential
frt=((math.sqrt(LG*gamR))/(f**2))-0.45        #Term with the frequency
vr=(1)/(1+sqt*((ext)*(frt)))
print('Vertical Reduction Factor= '+str(vr))

#Derive effective path length (STEP EIGHT)
LE=LR*vr  #in km
if LE<=0:
    LE=LE*-1
else:
    LE*1
print('Effective Path Length= '+str(LE)+'km')

#Calculate the predicated attenuation (STEP NINE)
Ap=gamR*LE
print('Predicted Attenuation= '+str(Ap)+'dB')
if elev>=25 or thet<36:
    z=-0.005*((thet)-(36))
elif elev<25 and thet<36:
    z=-0.005*(thet-36)+1.8-4.25*(math.sin(math.radians(elev)))
    
else:
    z=0 
    
#Attenuation Approximation for other percentages exceeded in the a range 0.001% -10% (STEP 10)    
def time(p):
    pro=[]
    for pr in p:
        lt=((0.045*math.log(Ap))-((z*(math.sin(math.radians(elev))))*(1-pr)))
        stt=(0.655+0.033*(math.log(pr)))
        Aps=Ap*((pr)/0.01)**(-1*((stt)-(lt)))
        Aps=10*(math.log10(Aps))
        Aps=round(Aps,4)
        pro.append(Aps)
    return pro
p=[1,0.5,0.1,0.01,0.001]
time(p)
print('Atmospheric Attenuation for 1%, 0.5%, 0.1%, 0.01% and 0.001% Unavailability Time Respectively= '+str(time(p))+ ' dB')


Enter the Ground Station Absolute Latitude: 25.88889
Enter Mean Annual 0° C Isotherm Height Above Mean Sea Level : 4.5
Rainfall Height= 4.86m
Enter the Ground Station Height above Sea Level in metres: 1385
Enter the Antenna Elevation Angle in degrees: 20
Slant Path Length= 4035.259404841084m
Enter 1-min rain rate exceeded for 0.01% of an average year from ITU-R P.837-7, raingauge or Satellite: 0.0668
Enter Coefficient k from RECOMMENDATION ITU-R P.838-3 : 0.2044
Enter Coefficient alpha from RECOMMENDATION ITU-R P.838-3 : 0.9314
Specific Attenuation= 0.01643908680103571dB/km
Enter the Transmission Frequency in GHz: 28.5
Horizontal Adjustement Factor= 1.61280833356096
Adjusted Rainy Path Length= -4035.259404841084m
Vertical Reduction Factor= -2.8689559659418147
Effective Path Length= 11576.981543641643km
Predicted Attenuation= 190.31500448991335dB
Atmospheric Attenuation for 1%, 0.5%, 0.1%, 0.01% and 0.001% Unailability Time Respectively= [14.4185, 15.921, 19.2109, 22.7947, 24.876] dB
